In [2]:
import sys
import pandas as pd
from typing import List, Dict
from SPARQLWrapper import SPARQLWrapper, JSON

class WikiDataQueryResults:
    """
    A class that can be used to query data from Wikidata using SPARQL and return the results as a Pandas DataFrame or a list
    of values for a specific key.
    """
    def __init__(self, query: str):
        """
        Initializes the WikiDataQueryResults object with a SPARQL query string.

        :param query: A SPARQL query string.
        """
        self.user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
        self.endpoint_url = "https://query.wikidata.org/sparql"
        self.sparql = SPARQLWrapper(self.endpoint_url, agent=self.user_agent)
        self.sparql.setQuery(query)
        self.sparql.setReturnFormat(JSON)

    def __transform2dicts(self, results: List[Dict]) -> List[Dict]:
        """
        Helper function to transform SPARQL query results into a list of dictionaries.

        :param results: A list of query results returned by SPARQLWrapper.
        :return: A list of dictionaries, where each dictionary represents a result row and has keys corresponding to the
        variables in the SPARQL SELECT clause.
        """
        new_results = []
        for result in results:
            new_result = {}
            for key in result:
                new_result[key] = result[key]['value']
            new_results.append(new_result)
        return new_results

    def _load(self) -> List[Dict]:
        """
        Helper function that loads the data from Wikidata using the SPARQLWrapper library, and transforms the results into
        a list of dictionaries.

        :return: A list of dictionaries, where each dictionary represents a result row and has keys corresponding to the
        variables in the SPARQL SELECT clause.
        """
        results = self.sparql.queryAndConvert()['results']['bindings']
        results = self.__transform2dicts(results)
        return results

    def load_as_dataframe(self) -> pd.DataFrame:
        """
        Executes the SPARQL query and returns the results as a Pandas DataFrame.

        :return: A Pandas DataFrame representing the query results.
        """
        results = self._load()
        return pd.DataFrame.from_dict(results)

In [4]:
query = """#Places of residence of accused witches in Scotland 1563-1736
SELECT ?item ?witch ?itemLabel ?residenceLabel ?coordinate ?genderLabel ?occupationLabel ?chargeLabel ?classLabel ?ethnicLabel ?mdeathLabel ?cdeathLabel ?deathlocLabel ?detainLabel
WHERE {
  ?item wdt:P4478 ?witch;
    wdt:P551 ?residence.
  ?residence wdt:P625 ?coordinate.
  OPTIONAL 
  {
    ?item wdt:P21 ?gender .
  }
  OPTIONAL
  {
    ?item wdt:P106 ?occupation .
  }
  OPTIONAL 
  {
    #?item wdt:P1595 ?charge . 
  }
  OPTIONAL
  {
    ?item wdt:P3716 ?class . 
  }
  OPTIONAL 
  {
    ?item wdt:P172 ?ethnic .
  }
  OPTIONAL 
  {
    ?item wdt:P1196 ?mdeath . 
  } 
  OPTIONAL 
  { 
    #?item wdt:P509 ?cdeath .
  }
  OPTIONAL 
  {
    ?item wdt:P20 ?deathloc . 
   } 
  OPTIONAL 
  {
    ?item wdt:P2632 ?detain . 
  }
 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""

data_extracter = WikiDataQueryResults(query)
df = data_extracter.load_as_dataframe()

                                       item      witch  \
0  http://www.wikidata.org/entity/Q43389933  A/EGD/947   
1  http://www.wikidata.org/entity/Q43390990  A/EGD/385   
2  http://www.wikidata.org/entity/Q43390997  A/EGD/391   
3  http://www.wikidata.org/entity/Q43391058  A/EGD/443   
4  http://www.wikidata.org/entity/Q43391069  A/EGD/454   

                   coordinate        itemLabel residenceLabel genderLabel  \
0      Point(-3.6293 56.0554)     Helene Ezatt        Culross      female   
1     Point(-3.14673 55.9416)    Issobel Broun    Duddingston      female   
2     Point(-3.14673 55.9416)    Isobel Ramsay    Duddingston      female   
3  Point(-3.783964 57.583936)  Elspet Falconer         Penick      female   
4  Point(-3.753976 57.556804)        Janet Man    East Milton      female   

  detainLabel classLabel         mdeathLabel deathlocLabel occupationLabel  \
0     Culross        NaN                 NaN           NaN             NaN   
1         NaN   middling        

In [17]:
df[df['mdeathLabel'].notnull()]

,item,witch,coordinate,itemLabel,residenceLabel,genderLabel,detainLabel,classLabel,mdeathLabel,deathlocLabel,occupationLabel,ethnicLabel
2,http://www.wikidata.org/entity/Q43390997,A/EGD/391,Point(-3.14673 55.9416),Isobel Ramsay,Duddingston,female,NaN,NaN,capital punishment,NaN,NaN,NaN
14,http://www.wikidata.org/entity/Q43391292,A/EGD/598,Point(-3.6293 56.0554),Katherine Sands,Culross,female,Culross,NaN,capital punishment,Gallow Lea,NaN,NaN
32,http://www.wikidata.org/entity/Q43394946,A/EGD/377,Point(-3.13083 55.905),Elspett Blackie,Gilmerton,female,NaN,NaN,capital punishment,Gilmerton,NaN,NaN
37,http://www.wikidata.org/entity/Q43395018,A/EGD/46,Point(-3.13083 55.905),Meg Dow,Gilmerton,female,NaN,NaN,capital punishment,Castle Hill,NaN,NaN
40,http://www.wikidata.org/entity/Q43395045,A/EGD/51,Point(-2.961 55.9597),Johne McGill (McGillis),Prestonpans,male,NaN,NaN,capital punishment,Haddington,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3163,http://www.wikidata.org/entity/Q43395326,A/EGD/90,Point(-2.78332 55.95612),Meg Dun,Haddington,female,NaN,NaN,capital punishment,Haddington,NaN,NaN
3169,http://www.wikidata.org/entity/Q43393935,A/EGD/1764,Point(-4.545491 55.902521),John Lindsay,Formakin,male,Glasgow,NaN,capital punishment,Gallows Green,NaN,NaN
3170,http://www.wikidata.org/entity/Q43393935,A/EGD/1764,Point(-4.545491 55.902521),John Lindsay,Formakin,male,Paisley,NaN,capital punishment,Gallows Green,NaN,NaN
3171,http://www.wikidata.org/entity/Q43393935,A/EGD/1764,Point(-4.545491 55.902521),John Lindsay,Formakin,male,Renfrew,NaN,capital punishment,Gallows Green,NaN,NaN


In [15]:
df[df['detainLabel'].notnull()]['detainLabel'].drop_duplicates()

0                  Culross
12                  Forfar
13           Kirkcudbright
20       Ross and Cromarty
23               Kirkcaldy
               ...        
2753    Berwick-upon-Tweed
2853            Mid Calder
2922              Galloway
2923          New Galloway
3069        Orkney Islands
Name: detainLabel, Length: 96, dtype: object

In [6]:
df[df['itemLabel']=='Niniane Chirneyside']

,item,witch,coordinate,itemLabel,residenceLabel,genderLabel,detainLabel,classLabel,mdeathLabel,deathlocLabel,occupationLabel,ethnicLabel
620,http://www.wikidata.org/entity/Q43392555,A/EGD/102,Point(-3.189166666 55.953333333),Niniane Chirneyside,Edinburgh,male,NaN,middling,NaN,NaN,servant,NaN
